In [48]:
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
from geopy.geocoders import Nominatim
geolocator = Nominatim()


In [49]:
# Read in File

students = pd.read_pickle('students')
#students.head()

In [50]:
# Cleaning
# get rid of negative time 
#students = students[students['time_enrolled'] >= 0]
enr_d = students[students['time_enrolled'] < 0]['enroll_date']
end_d = students[students['time_enrolled'] < 0]['end_date']

students.enroll_date[students['time_enrolled'] < 0] = end_d
students.end_date[students['time_enrolled'] < 0] = enr_d
students.time_enrolled[students['time_enrolled'] < 0] = 102
#((enr_d-end_d)/np.timedelta64(1, 'D')).astype(int)
students.time_enrolled[students['time_enrolled'] < 0] 


/home/harisk87/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/harisk87/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/harisk87/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Series([], Name: time_enrolled, dtype: int64)

In [51]:
# distribution of times

trace1 = Histogram(x=students['time_enrolled'], marker=dict(color='rgb(0, 128, 128)'))
data = [trace1]
layout = Layout(xaxis=dict(autotick=False,tick0=0,dtick=1000, tickfont=dict(size=18),title = 'Time of Enrollment (Days)', titlefont=dict(size=24)),yaxis=dict(autotick=False,tick0=0,dtick=100, tickfont=dict(size=18), title='Number of students', titlefont=dict(size=24), showgrid=False),title='Distribution of Time of Enrollment for students (Days)',titlefont=dict(size=24))


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

In [52]:
print(len(students))
students['time_enrolled'].median()

2658


230.0

In [53]:
# another definition of short and long term

ndays = 120
students['shorter_term'] = 0
students.ix[students['time_enrolled'] <= ndays, 'shorter_term'] = 1


In [54]:
# More cleaning : distance and empty rows

studentsshort = students[students['dist'] != ""]
print(len(studentsshort))
studentsshortnew = studentsshort[studentsshort['dist']<100]
print(len(studentsshortnew))
print(studentsshortnew['dist'].max())

trace1 = Histogram(x=studentsshortnew['dist'])
data = [trace1]
layout = Layout(xaxis=dict(title = 'Distance (miles)'),yaxis=dict(title='Number of students'),title='Distribution of Distance between camp and student (miles)')
fig1 = Figure(data=data, layout=layout)
iplot(fig1)


studentsshortnew['dist_norm'] = studentsshortnew['dist'] / studentsshortnew['dist'].max()
# for now normalize by max,change later
studentsshortnew.head()



2611
2242
50.3791061577


/home/harisk87/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,center_days,time_enrolled,short_term,age_2016,age_enroll,age_end,days_open,dist,shorter_term,dist_norm
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,"[2, 5]",90,1,14,13,13,2,5.16363,1,0.102496
2,2824,61938,M,2000-12-13,2015-01-01,2015-08-30,12,Santa Rosa,USA,95404,...,"[2, 5]",241,0,15,14,14,2,2.46161,0,0.0488616
4,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,"[2, 5]",85,1,8,8,8,2,5.16363,1,0.102496
5,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,"[2, 5]",83,1,11,11,11,2,5.16363,1,0.102496
9,2824,62074,M,2000-06-28,2015-05-01,2015-09-27,17,Rohnert Park,USA,94928,...,"[2, 5]",149,1,16,14,15,2,9.30968,0,0.184793


In [55]:
# Census Data

income1 = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/Insight-Project/IncomeData.csv')
income = income1[['GEO.id2', 'HC02_EST_VC02', 'HC02_EST_VC04']]
income = income.rename(columns={'GEO.id2':'zip_student'})
# get zip codes and median income
studentsshortnew = pd.merge(studentsshortnew, income, on='zip_student')
# put in median income into clumns
studentsshortnew = studentsshortnew.rename(columns={'HC02_EST_VC02': 'household_income', 'HC02_EST_VC04': 'median_income'})



studentsshortnew['median_income'] = pd.to_numeric(studentsshortnew['median_income'], errors='coerce')
studentsshortnew['household_income'] = pd.to_numeric(studentsshortnew['household_income'], errors='coerce')

studentsshortnew = studentsshortnew.dropna(how='any')
print(len(studentsshortnew))
studentsshortnew['median_income_norm'] = studentsshortnew['median_income']/studentsshortnew['median_income'].max()
studentsshortnew['household_income_norm'] = studentsshortnew['household_income']/studentsshortnew['household_income'].max()

studentsshortnew.head()

2233


,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,age_enroll,age_end,days_open,dist,shorter_term,dist_norm,household_income,median_income,median_income_norm,household_income_norm
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,13,13,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,8,8,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,11,11,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,9,10,2,5.16363,0,0.102496,65453.0,65505.0,0.30956,0.298022
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,7,7,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022


In [56]:
# Add scores

scores = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/EduCare/ScoresCA.csv')
scores1 = scores.groupby(['student_id'], sort=False)['grade_level'].max()
scores2 = scores.groupby(['student_id'], sort=False)['grade_level'].min()
scores3 = pd.DataFrame({'student_id':scores1.keys(), 'grade_level_end':scores1.values})
scores4 = pd.DataFrame({'student_id':scores2.keys(), 'grade_level_begin':scores2.values})
studentsn = pd.merge(studentsshortnew, scores4, on='student_id')
studentsn = pd.merge(studentsn, scores3, on='student_id')
studentsn.head()

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,days_open,dist,shorter_term,dist_norm,household_income,median_income,median_income_norm,household_income_norm,grade_level_begin,grade_level_end
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022,11,12
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022,5,7
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022,6,7
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,2,5.16363,0,0.102496,65453.0,65505.0,0.30956,0.298022,2,9
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,2,5.16363,1,0.102496,65453.0,65505.0,0.30956,0.298022,5,6


In [57]:
studentsn['skill_increase'] = ((studentsn['grade_level']-studentsn['grade_level_begin'])/studentsn['time_enrolled'])
studentsn['skill_increase_correct'] = ((studentsn['grade_level_end']-studentsn['grade_level_begin'])/studentsn['time_enrolled'])
len(studentsn)

2233

In [58]:
studentsn = studentsn.dropna(how='any')
studentsn = studentsn.replace([np.inf, -np.inf], np.nan).dropna(subset=["skill_increase"], how="all")
studentsn = studentsn.replace([np.inf, -np.inf], np.nan).dropna(subset=["skill_increase_correct"], how="all")
studentsn['skill_increase_norm'] = studentsn['skill_increase']/studentsn['skill_increase'].max()
studentsn['skill_increase_norm_correct'] = studentsn['skill_increase_correct']/studentsn['skill_increase_correct'].max()

print(len(studentsn))
studentsn.head()
#scores[scores['student_id'] == 63435]
#studentsn[studentsn['skill_increase_correct'] != studentsn['skill_increase']]#.min()


2229


,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,household_income,median_income,median_income_norm,household_income_norm,grade_level_begin,grade_level_end,skill_increase,skill_increase_correct,skill_increase_norm,skill_increase_norm_correct
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,65453.0,65505.0,0.30956,0.298022,11,12,0.011111,0.011111,0.008333,0.008333
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,65453.0,65505.0,0.30956,0.298022,5,7,0.023529,0.023529,0.017647,0.017647
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,65453.0,65505.0,0.30956,0.298022,6,7,0.012048,0.012048,0.009036,0.009036
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,65453.0,65505.0,0.30956,0.298022,2,9,0.034188,0.029915,0.025641,0.022436
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,65453.0,65505.0,0.30956,0.298022,5,6,0.043478,0.043478,0.032609,0.032609


In [59]:
#studentsn.columns

# Features: gender (not great predictor), Age of enrollment, Skill Level, 
# City camp is in (replacing with continuous feature),  Distance of student from camp based on zip codes (norm), 
# days center is open, household or median income (norm), beginnig skill level (not great predictor), 
# rate of increase of skill level correct (normalized)

# eventually save another pickle file for students

studentsn['age_enroll_norm'] = studentsn['age_enroll']/studentsn['age_enroll'].max()
studentsn['grade_level_begin_norm'] = studentsn['grade_level_begin']/studentsn['grade_level_begin'].max()
studentsn['grade_level_end_norm'] = studentsn['grade_level_end']/studentsn['grade_level_end'].max()

In [60]:
studentsn['gender_norm'] = 0
studentsn.ix[studentsn['gender'] == 'F', 'gender_norm'] = 1
studentsn.head()

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,grade_level_begin,grade_level_end,skill_increase,skill_increase_correct,skill_increase_norm,skill_increase_norm_correct,age_enroll_norm,grade_level_begin_norm,grade_level_end_norm,gender_norm
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,11,12,0.011111,0.011111,0.008333,0.008333,0.282609,0.6875,0.571429,0
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,5,7,0.023529,0.023529,0.017647,0.017647,0.173913,0.3125,0.333333,1
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,6,7,0.012048,0.012048,0.009036,0.009036,0.239130,0.3750,0.333333,1
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,2,9,0.034188,0.029915,0.025641,0.022436,0.195652,0.1250,0.428571,1
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,5,6,0.043478,0.043478,0.032609,0.032609,0.152174,0.3125,0.285714,1


In [236]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression(penalty = "l2", C=1000000, class_weight = 'balanced', tol = 0.0001)
#feature_cols = ['age_enroll','days_open', 'dist_norm', 'median_income_norm',  'grade_level_end','grade_level_begin', 'skill_increase_norm_correct']

#feature_cols = ['age_enroll_norm', 'grade_level_begin_norm', 'days_open', 'gender_norm', 'skill_increase_norm_correct']
#vv = ['Age Enrolled', 'Rate of Increase of Skill Level', 'Begin Skill Level']


#feature_cols = ['dist_norm','median_income_norm', 'age_enroll','days_open',  'grade_level_end','grade_level_begin', 'gender_norm']
#vv = ['Distance', 'Median Income' , 'Age Enrolled', 'Days Open', 'Ending Skill Level','Begin Skill Level', 'Gender']


#feature_cols = ['age_enroll','days_open',  'grade_level_end','grade_level_begin', 'gender_norm']
#vv = ['Age Enrolled', 'Days Open', 'End Skill Level','Begin Skill Level', 'Gender']


feature_cols = ['dist_norm','median_income_norm' ,'age_enroll_norm', 'grade_level_begin_norm',  'grade_level_end_norm']
vv = ['Distance', 'Median Income', 'Age Enrolled', 'Begin Skill Level', 'End Skill Level']

#feature_cols = ['age_enroll_norm', 'skill_increase_norm_correct', 'grade_level_begin_norm',  'grade_level_end_norm']
#vv = ['Age Enrolled', 'Growth', 'Begin Skill Level', 'End Skill Level']

xprev = studentsn[feature_cols]#.reset_index()[feature_cols]
y = studentsn['short_term']#.reset_index()['short_term']x

#xprev = studentsn[studentsn['time_enrolled'] < 1000][feature_cols] # without outliers - same importance and accuracy
#y = studentsn[studentsn['time_enrolled'] < 1000]['short_term']

x = xprev



X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([i for i in y_test if i == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
#vv = list(x)

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

def crossValidate(model,param_grid,X,y,scores):

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        grid = GridSearchCV(model, param_grid, cv=5, scoring=score)
        grid.fit(X_train,y_train)

        print("Best parameters set found on development set:")
        print()
        print(grid.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        for params, mean_score, scores in grid.grid_scores_:
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean_score, scores.std() * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the training set.")
        print("The scores are computed on the test set.")
        print()
        y_true, y_pred = y_test, grid.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()
logreg = LogisticRegression(penalty = "l2", class_weight = 'balanced', tol = 0.0001)
Cvals = [0.0000000000001,0.1,1,10,100,1000,10000,1000000,1000000,10000000,100000000,100000000000000000000000000000]
param_grid = dict(C=Cvals)
scores = ['accuracy']
crossValidate(logreg,param_grid,x,y,scores)

#print(fimplr.ravel())
#print(yv)
#print(yv/yv.max())

yv = abs(fimplr.ravel())
yvv = yv/yv.max()

inds = yvv.argsort()[::-1]
print(inds)
print(vv)
vvv = np.array(vv)
print(vvv)
sortvv = vvv[inds]
print(sortvv)
#[x for y, x in sorted(zip(Y, X))]

data = [Bar(x=sortvv,y=np.sort(yvv)[::-1], marker=dict(color='rgb(0, 128, 128)'))]

layout = Layout(xaxis=dict(tickfont=dict(size=20)),
                 yaxis=dict(title='Coefficients (Normalized)', showgrid=False, tickfont=dict(size=20), titlefont=dict(size=24)),
                title='Feature Coefficients', titlefont=dict(size=24))


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


#inds = ages.argsort()
#sortedPeople = people[inds]




data = [Bar(x=vv,y=abs(fimplr.ravel()))]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title=' Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#print(fimplr.ravel())
yv = abs(fimplr.ravel())
#print(yv)
#print(np.log(yv))



data = [Bar(x=vv,y=np.log(yv))]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Log Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)










Accuracy of model on test set is 80.0448430493 %
F1 scores are long-term/short-term[ 0.8         0.80089485]
Precision scores are lt-st[ 0.80542986  0.79555556]
Recall scores are lt-st[ 0.79464286  0.80630631]
[[178  46]
 [ 43 179]]
Predicting everything as falling as long term will give accuracy of 50.2242152466
# Tuning hyper-parameters for accuracy
()
Best parameters set found on development set:
()
{'C': 1000}
()
Grid scores on development set:
()
0.494 (+/-0.001) for {'C': 1e-13}
0.777 (+/-0.030) for {'C': 0.1}
0.801 (+/-0.038) for {'C': 1}
0.807 (+/-0.033) for {'C': 10}
0.809 (+/-0.031) for {'C': 100}
0.811 (+/-0.030) for {'C': 1000}
0.811 (+/-0.030) for {'C': 10000}
0.811 (+/-0.030) for {'C': 1000000}
0.811 (+/-0.030) for {'C': 1000000}
0.811 (+/-0.030) for {'C': 10000000}
0.811 (+/-0.030) for {'C': 100000000}
0.811 (+/-0.030) for {'C': 100000000000000000000000000000L}
()
Detailed classification report:
()
The model is trained on the training set.
The scores are computed on the 

In [135]:
#Feature Engineering and Accuracy changes
#
ltsk = studentsn[studentsn['short_term']==0]['skill_increase_correct']
len(ltsk)

1104

In [136]:
import scipy as sp
def cohensd(val1,val2):
    return ( np.mean(val1) - np.mean(val2) ) / np.sqrt( (np.std(val1) ** 2 + np.std(val2) ** 2) / 2 )

stsk = studentsn[studentsn['short_term']==1]['skill_increase_correct']
ltsk = studentsn[studentsn['short_term']==0]['skill_increase_correct']
print("The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms increase skills on average at rate " + str(np.median(stsk)))
print("LOng terms increase skills on avergae at rate " + str(np.median(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['grade_level_begin']
ltsk = studentsn[studentsn['short_term']==0]['grade_level_begin']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms begin at grade level " + str(np.mean(stsk)))
print("LOng terms begin at grade level " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['dist']
ltsk = studentsn[studentsn['short_term']==0]['dist']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms commute " + str(np.mean(stsk)))
print("LOng terms commute " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['median_income']
ltsk = studentsn[studentsn['short_term']==0]['median_income']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms income " + str(np.mean(stsk)))
print("LOng terms income " + str(np.mean(ltsk)))
stsk = studentsn[studentsn['short_term']==1]['age_enroll']
ltsk = studentsn[studentsn['short_term']==0]['age_enroll']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms enroll age " + str(np.median(stsk)))
print("LOng terms enroll age " + str(np.median(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['grade_level_begin']
ltsk = studentsn[studentsn['short_term']==0]['grade_level_begin']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms entry level " + str(np.median(stsk)))
print("LOng terms entry level " + str(np.median(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['grade_level_end']
ltsk = studentsn[studentsn['short_term']==0]['grade_level_end']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms exit level " + str(np.median(stsk)))
print("LOng terms exit level " + str(np.median(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")

stsk = studentsn[studentsn['short_term']==1]['days_open']
ltsk = studentsn[studentsn['short_term']==0]['days_open']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms days open " + str(np.mean(stsk)))
print("LOng terms days open " + str(np.mean(ltsk)))

The p value is 2.84121534052e-28 and the t value is 5.6305342423
The effect size is 0.475059386294
SHort terms increase skills on average at rate 0.01875
LOng terms increase skills on avergae at rate 0.00796020688617

 The p value is 0.0127760178808 and the t value is 1.11723781798
The effect size is 0.0947476616482
SHort terms begin at grade level 6.14133333333
LOng terms begin at grade level 5.90760869565

 The p value is 0.310625146601 and the t value is 0.247074074105
The effect size is 0.0209420240341
SHort terms commute 3.10984037795
LOng terms commute 3.03465083303

The p value is 0.422897904187 and the t value is 0.0972540495816
The effect size is 0.00824202570695
SHort terms income 88695.1208889
LOng terms income 88467.8097826

The p value is 7.01374482192e-11 and the t value is 3.22366082497
The effect size is 0.272972052696
SHort terms enroll age 9.0
LOng terms enroll age 8.0

The p value is 0.0127760178808 and the t value is 1.11723781798
The effect size is 0.0947476616482


In [64]:
ltsk = studentsn[studentsn['short_term']==1]['skill_increase_correct']

trace2 = Histogram(x=ltsk,opacity=0.75, name = 'Long Term Enrollees')
data = [ trace2]
layout = Layout(barmode='overlay',xaxis=dict(title = 'Rate of Skill Increase (per day)'),yaxis=dict(title='Number of students'),
                title='Comparing Rate of Skill Increase for Long and Short Term Enrollees')
fig = Figure(data=data, layout=layout)
iplot(fig)
len(ltsk)

1125

In [235]:
# 2 distributions

stsk = studentsn[studentsn['short_term']==1]['skill_increase_correct']
ltsk = studentsn[studentsn['short_term']==0]['skill_increase_correct']

trace1 = Histogram(x=stsk,opacity=0.75, name='Short Term Enrollees')
trace2 = Histogram(x=ltsk,opacity=0.75, name = 'Long Term Enrollees')
data = [trace1, trace2]
layout = Layout(barmode='overlay',xaxis=dict(title = 'Rate of Skill Increase (per day)'),yaxis=dict(title='Number of students'),
                title='Comparing Rate of Skill Increase for Long and Short Term Enrollees')
fig = Figure(data=data, layout=layout)
iplot(fig)

#
stsk = studentsn[studentsn['short_term']==1]['grade_level_end']
ltsk = studentsn[studentsn['short_term']==0]['grade_level_end']

trace1 = Histogram(x=stsk,opacity=0.75, name='Short-Term Enrollees', marker=dict(color='rgb(125, 70, 200)'))
trace2 = Histogram(x=ltsk,opacity=0.75, name = 'Long Term Enrollees',  marker=dict(color='rgb(250, 130, 170)'))
data = [trace1, trace2]

layout = Layout(barmode='overlay',xaxis=dict(title = 'End Skill Level', tickfont=dict(size=20), titlefont=dict(size=24)),yaxis=dict(title='Number of Students', showgrid=False,tickfont=dict(size=20),dtick=40, titlefont=dict(size=24)),
                title='Distribution of End Skill Level for Short and Long Term Students', titlefont=dict(size=24), showlegend=False)
fig = Figure(data=data, layout=layout)
iplot(fig)



stsk = studentsn[studentsn['short_term']==1]['age_enroll']
ltsk = studentsn[studentsn['short_term']==0]['age_enroll']

trace1 = Histogram(x=stsk,opacity=0.75, name='Short Term Enrollees')
trace2 = Histogram(x=ltsk,opacity=0.75, name = 'Long Term Enrollees')
data = [trace1, trace2]
layout = Layout(barmode='overlay',xaxis=dict(title = 'Age of Student (years)'),yaxis=dict(title='Number of students'),
                title='Comparing Age of Long and Short Term Enrollees')
fig = Figure(data=data, layout=layout)
iplot(fig)

In [145]:
# Insight: SKill level increase
# SKill level exit

trace1 = Scatter(x=studentsn['skill_increase_correct'],y=studentsn['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Rate of Increase of Skill (per day)'),
                 yaxis=dict(title='Time Enrolled (days)'),
                title='Rate of Increase in Skill vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

xx = studentsn.groupby('grade_level_end')['time_enrolled'].mean().keys()
yy = studentsn.groupby('grade_level_end')['time_enrolled'].mean().values
er = studentsn.groupby('grade_level_end')['time_enrolled'].std().values#/studentsn.groupby('grade_level_end')['time_enrolled'].size().values

data = [Scatter(x=xx,y=yy,error_y=dict(type='data',array=er,visible=True))]
layout = Layout(xaxis=dict(title = 'End Skill Level'),
                 yaxis=dict(title='Time Enrolled (days)'),
                title='End Skill Level vs Time Enrolled')

fig1 = Figure(data=data, layout=layout)
iplot(fig1)














stsk = studentsn[studentsn['short_term']==1]['grade_level_end']
stsk1 = studentsn[studentsn['short_term']==1]['time_enrolled']

ltsk = studentsn[studentsn['short_term']==0]['grade_level_end']
ltsk1 = studentsn[studentsn['short_term']==0]['time_enrolled']

trace1 = Scatter(x=stsk,y=stsk1, mode='markers', name = 'short-term')
trace2 = Scatter(x=ltsk,y=ltsk1, mode='markers', name = 'long-term')

data = [trace1,trace2]
layout = Layout(xaxis=dict(title = 'Ending grade level'),
                 yaxis=dict(title='Time Enrolled (days)'),
                title='Ending grade level vs Time Enrolled')

fig1 = Figure(data=data, layout=layout)
iplot(fig1)

In [228]:
stsk = studentsn[studentsn['short_term']==1]['skill_increase_correct']
stsk1 = studentsn[studentsn['short_term']==1]['time_enrolled']

ltsk = studentsn[studentsn['short_term']==0]['skill_increase_correct']
ltsk1 = studentsn[studentsn['short_term']==0]['time_enrolled']

trace1 = Scatter(x=stsk,y=stsk1, mode='markers', name = 'Short-Term Enrollees', marker=dict(color='rgb(100, 100, 200)', size=5))
trace2 = Scatter(x=ltsk,y=ltsk1, mode='markers', name = 'Long-Term Enrollees', marker=dict(color='rgb(250, 130, 170)',size=5))

#data = [trace1,trace2]
#layout = Layout(xaxis=dict(title = 'Rate of Increase of Skill (per day)',range=[0,0.2]),
#                 yaxis=dict(title='Time Enrolled (days)'),
#                title='Rate of Increase in Skill vs Time Enrolled')

#fig1 = Figure(data=data, layout=layout)
#iplot(fig1)

#data = [Bar(x=sortvv,y=np.sort(yvv)[::-1], marker=dict(color='rgb(0, 128, 128)'))]

#l
# Log Plot

data = [trace1,trace2]
layout = Layout(xaxis=dict(title = 'Growth (per Day)',type='log', autorange='True',tickfont=dict(size=20), showgrid=False,  titlefont=dict(size=24)),
                 yaxis=dict(title='Time Enrolled (Days)', showgrid=False, tickfont=dict(size=20), titlefont=dict(size=24),dtick=1000),
                title='Growth of Student vs Time Enrolled',  titlefont=dict(size=24), showlegend=False)
fig1 = Figure(data=data, layout=layout)
iplot(fig1)

In [175]:
#studentsn[studentsn['grade_level_end'] == 5]['time_enrolled'].min()
studentsn.columns

Index([u'camp_id', u'student_id', u'gender', u'date_of_birth', u'enroll_date',
       u'end_date', u'grade_level', u'city_student', u'country_student',
       u'zip_student', u'state_student', u'city_camp', u'zip_camp',
       u'state_camp,', u'country_camp', u'open_date', u'center_days',
       u'time_enrolled', u'short_term', u'age_2016', u'age_enroll', u'age_end',
       u'days_open', u'dist', u'shorter_term', u'dist_norm',
       u'household_income', u'median_income', u'median_income_norm',
       u'household_income_norm', u'grade_level_begin', u'grade_level_end',
       u'skill_increase', u'skill_increase_correct', u'skill_increase_norm',
       u'skill_increase_norm_correct', u'age_enroll_norm',
       u'grade_level_begin_norm', u'grade_level_end_norm', u'gender_norm'],
      dtype='object')

In [140]:
stsk = studentsn[studentsn['short_term']==1]#['skill_increase_correct']
ltsk = studentsn[studentsn['short_term']==0]#['skill_increase_correct']
ltsk

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,grade_level_begin,grade_level_end,skill_increase,skill_increase_correct,skill_increase_norm,skill_increase_norm_correct,age_enroll_norm,grade_level_begin_norm,grade_level_end_norm,gender_norm
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,2,9,0.034188,0.029915,0.025641,0.022436,0.195652,0.1250,0.428571,1
6,2824,63963,M,2005-06-04,2015-05-04,2016-03-26,9,Santa Rosa,USA,95409,...,5,9,0.012232,0.012232,0.009174,0.009174,0.195652,0.3125,0.428571,0
7,2825,477782,M,2006-01-04,2011-02-03,2013-06-30,8,Mill Valley,USA,95409,...,2,8,0.006834,0.006834,0.005125,0.005125,0.108696,0.1250,0.380952,0
8,2825,478329,F,2009-03-18,2014-06-12,2015-12-21,7,Mill Valley,USA,95409,...,2,7,0.008977,0.008977,0.006732,0.006732,0.108696,0.1250,0.333333,1
9,2824,61938,M,2000-12-13,2015-01-01,2015-08-30,12,Santa Rosa,USA,95404,...,8,12,0.016598,0.016598,0.012448,0.012448,0.304348,0.5000,0.571429,0
10,2824,62156,F,2003-01-14,2015-01-01,2016-04-14,15,Santa Rosa,USA,95404,...,6,15,0.019190,0.019190,0.014392,0.014392,0.239130,0.3750,0.714286,1
11,2824,62196,F,2006-03-24,2015-06-16,2016-09-03,8,SANTA ROSA,USA,95404,...,5,8,0.006742,0.006742,0.005056,0.005056,0.195652,0.3125,0.380952,1
13,2824,62792,M,2006-12-04,2015-01-01,2016-08-21,11,Santa Rosa,USA,95404,...,5,12,0.010033,0.011706,0.007525,0.008779,0.173913,0.3125,0.571429,0
20,2824,63640,M,2007-12-03,2015-01-01,2015-08-21,9,Rohnert Park,USA,94928,...,2,9,0.030172,0.030172,0.022629,0.022629,0.152174,0.1250,0.428571,0
30,2824,62379,F,2007-11-13,2015-01-01,2016-06-06,8,Windsor,USA,95492,...,5,8,0.005747,0.005747,0.004310,0.004310,0.152174,0.3125,0.380952,1


In [92]:
np.corrcoef(studentsn['time_enrolled'],studentsn['age_enroll_norm'])

array([[ 1.        , -0.16518928],
       [-0.16518928,  1.        ]])

In [90]:
studentsn.columns

Index([u'camp_id', u'student_id', u'gender', u'date_of_birth', u'enroll_date',
       u'end_date', u'grade_level', u'city_student', u'country_student',
       u'zip_student', u'state_student', u'city_camp', u'zip_camp',
       u'state_camp,', u'country_camp', u'open_date', u'center_days',
       u'time_enrolled', u'short_term', u'age_2016', u'age_enroll', u'age_end',
       u'days_open', u'dist', u'shorter_term', u'dist_norm',
       u'household_income', u'median_income', u'median_income_norm',
       u'household_income_norm', u'grade_level_begin', u'grade_level_end',
       u'skill_increase', u'skill_increase_correct', u'skill_increase_norm',
       u'skill_increase_norm_correct', u'age_enroll_norm',
       u'grade_level_begin_norm', u'grade_level_end_norm', u'gender_norm'],
      dtype='object')

In [212]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10)

feature_cols = ['median_income_norm', 'age_enroll_norm', 'skill_increase_norm_correct', 'grade_level_begin_norm',  'grade_level_end_norm', 'dist_norm']

xprev = studentsn[feature_cols]#.reset_index()[feature_cols]
y = studentsn['short_term']#.reset_index()['short_term']x
x = xprev
vv = list(x)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
#print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = rf.feature_importances_
print(fimplr.ravel())

data = [Bar(x=vv,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


Accuracy of model on test set is 92.1524663677 %
[[209  11]
 [ 24 202]]
Predicting everything as falling as long term will give accuracy of 49.3273542601
[ 0.06877948  0.07670018  0.47661406  0.11448332  0.21051368  0.05290927]
